# HAPTICA Model Testing Notebook
Test and validate the gesture recognition model

In [ ]:
import sys
sys.path.append('../src')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from inference.predictor import GesturePredictor
from preprocessing.transforms import ImageTransforms

## Load Model and Test

In [ ]:
# Initialize components
model_path = "../models/hand_recognition_model.h5"
labels_path = "../config/labels.json"

predictor = GesturePredictor(model_path, labels_path)
transforms = ImageTransforms(target_size=(224, 224))

print("Model Info:")
print(predictor.get_model_info())

## Test with Sample Images

In [ ]:
# Test with sample images from dataset
test_images_dir = Path("../leapgestrecog/00/01_palm")

if test_images_dir.exists():
    image_files = list(test_images_dir.glob("*.png"))[:5]  # Test first 5 images
    
    fig, axes = plt.subplots(1, len(image_files), figsize=(15, 3))
    
    for i, img_path in enumerate(image_files):
        # Load and preprocess image
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Preprocess for model
        processed = transforms.preprocess_roi(img)
        
        if processed is not None:
            # Make prediction
            result = predictor.predict(processed)
            
            # Display
            axes[i].imshow(img_rgb)
            axes[i].set_title(f"{result['gesture']}\n{result['confidence']:.2f}")
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("Test images directory not found")

## Live Camera Test

In [ ]:
# Test with live camera (run this cell to test real-time prediction)
def test_live_camera():
    cap = cv2.VideoCapture(0)
    
    print("Press 'q' to quit live test")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Simple preprocessing (crop center region)
        h, w = frame.shape[:2]
        center_crop = frame[h//4:3*h//4, w//4:3*w//4]
        
        # Preprocess and predict
        processed = transforms.preprocess_roi(center_crop)
        if processed is not None:
            result = predictor.predict(processed)
            
            # Draw result on frame
            cv2.putText(frame, f"{result['gesture']}: {result['confidence']:.2f}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('HAPTICA Test', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Uncomment to run live test
# test_live_camera()